# Baseten <> Tensorflow example deployment

<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/deployment/baseten_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade tensorflow baseten

In [ ]:
# Model training

import tensorflow as tf

#Creates tensorflow model
def train_model():
    return tf.keras.applications.ResNet50V2(
        include_top=True,
        weights="imagenet",
        classifier_activation="softmax",
    )

In [ ]:
# Model creation

my_model = train_model()

In [ ]:
# Model packaging

import truss

tr = truss.create(my_model, "my-truss")
tr.add_python_requirement("scipy") # Needed for processing functions in next step

# Pre- and post-processing functions

The Tensorflow model has inputs and outputs that are not JSON-serializable. To make it so that we can use our deployed model over an API, we'll package pre- and post-processing functions with the model.

In your newly created Truss, open `my-truss/model/model.py` and add pre- and post-processing functions as follows.

First, add the following imports at the top of the file:


```python
import requests
import tempfile
import tensorflow as tf
from scipy.special import softmax
```

Then, update the pre-processing function to:

```python
def preprocess(self, model_input: Any) -> Any:
    """Preprocess step for ResNet"""
    request = requests.get(model_input)
    with tempfile.NamedTemporaryFile() as f:
        f.write(request.content)
        f.seek(0)
        input_image = tf.image.decode_png(tf.io.read_file(f.name))
    preprocessed_image = tf.keras.applications.resnet_v2.preprocess_input(
        tf.image.resize([input_image], (224, 224))
    )
    return np.array(preprocessed_image)
```

Finally, update the post-processing function to:

```python
def postprocess(self, model_output: Any, k=5) -> Any:
    """Post process step for ResNet"""
    class_predictions = model_output["predictions"][0]
    LABELS = requests.get(
        "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
    ).text.split("\n")
    class_probabilities = softmax(class_predictions)
    top_probability_indices = class_probabilities.argsort()[::-1][:k].tolist()
    return {
        LABELS[index]: 100 * class_probabilities[index].round(3)
        for index in top_probability_indices
    }
```

Save the file, and re-load the truss handle (`tr`) in the next cell to access the updated Truss.

In [ ]:
# Model deployment

import baseten

tr = truss.load("my-truss")

api_key = "PASTE API KEY HERE"
baseten.login(api_key)

baseten.deploy(
    tr,
    model_name="My ResNet TensorFlow Model"
)

In [ ]:
# After the deployment is finished, call your new model!

deployed_model_id = "PASTE VERSION ID HERE" # See deployed model page to find version ID
model_input = "https://github.com/pytorch/hub/raw/master/images/dog.jpg"

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

In [ ]:
# Or call your model as an API

import requests
import json

endpoint = f"https://app.baseten.co/model_versions/{deployed_model_id}/predict"
auth = {"Authorization": f"Api-Key {api_key}"}
data = json.dumps(model_input)

resp = requests.post(
    endpoint,
    headers=auth,
    data=data
)

resp.json()